In [1]:
collision_data = sqlContext.read.format('com.databricks.spark.csv').options(header='true',inferSchema='true').load('/FileStore/tables/mue9q9ad1469238369776/Collisions_Data.xlsx')


In [2]:
%sql
CREATE TABLE COLLISONS_L (
    lat string,  
  lon string,  
  object_id int,  
  case_id int,  
  acc_year int,  
  proc_data String,  
  juris int,  
  collision_data String,  
  coll_time int,  
  officer_id int,  
  reporting_district int,  
  day_of_week int,  
  shift int,  
  population int,  
  cnty_city_loc int,  
  datapoint float,  
  special_cond int,  
  beat_type int,  
  chp_beat_type int,  
  city_div string,  
  chp_beat_class int,  
  beat_numb string,  
  primary_rd string,  
  secondary_rd string,  
  distance int,  
  direction string,  
  intersection string,  
  weather_1 string,  
  weather_2 string,  
  state_hwy string,  
  caltrans_county string,  
  caltrans_dis string,  
  state_route int,  
  route_suffix string,  
  postmile string,  
  loc_type string,  
  ramp_inter string,  
  side_of_hwy string,  
  tow_awy string,  
  coll_sev int,  
  num_killed int,  
  num_inj int,  
  party_count int,  
  prim_coll_fac string,  
  pcf_code_vil string,  
  pcf_code_cat string,  
  pcf_vil string,  
  pcf_vil_sub string,  
  hit_run string,  
  type_of_coll string,  
  mviw string,  
  ped_action string,  
  road_surf string,  
  road_cond_1 string,  
  road_cond_2 string,  
  lighting string,  
  control_device string,  
  chp_row_type string,  
  pedesterian string,  
  bicycle string,  
  motorcycle string,  
  truck string,  
  not_private_prop string,  
  alchohol_ivolved string,  
  stwd_vec_type string,  
  chp_vec_type_fault string,  
  sev_inj_count string,  
  vis_inj_count string,  
  complain_plan_count string,  
  ped_killed_count string,  
  ped_inj_count string,  
  bicycle_killed_count string,  
  bicycle_killed_inj string,  
  motor_killed_count string,  
  motor_inj_count string,  
  primary_ramp string,  
  sec_ramp string,  
  latitude string,  
  longitude string,  
  time_cat int,  
  month string,  
  city string,  
  country string,  
  state string,  
  point_x string,  
  point_y string,  
  match_address string,  
  primary_yard string,  
  secondary_yard string  
 )
USING com.databricks.spark.csv
OPTIONS (path "/FileStore/tables/8qifqlgk1469240528424/Collisions_20092013_SWITRS__1_-d74ea.csv", header "true")



In [3]:
display(sqlContext.sql("SELECT LAT as long,LON as lat from COLLISONS_L"))

In [4]:
display(sqlContext.sql("SELECT acc_year FROM COLLISONS_L"))

number of collisions per year

In [6]:
display(sqlContext.sql("SELECT count(*),acc_year as year from COLLISONS_L group by acc_year"))

In [7]:
season_data=sqlContext.sql("select case when month between 1 and 3 then 'Winter' when month between 3 and 6 then 'Spring' when month between 6 and 7 then 'Summer' when month between 7 and 12 then 'Fall' end as month from COLLISONS_L where month is not null")

In [8]:
from pyspark.sql.functions import col
display(season_data.groupby('month').count().sort(col('month')))

coll during different times

In [10]:
time_data=sqlContext.sql("select case when coll_time between 0 and 200 then '00:00-2:00' when coll_time between 200 and 400 then '02:00-04:00' when coll_time between 400 and 600 then '04:00-06:00' when coll_time between 600 and 800 then '06:00-08:00' when coll_time between 800 and 1000 then '08:00-10:00' when coll_time between 1000 and 1200 then '10:00-12:00' when coll_time between 1200 and 1400 then '12:00-14:00' when coll_time between 1400 and 1600 then '14:00-16:00' when coll_time between 1600 and 1800 then '16:00-18:00' when coll_time between 1800 and 2000 then '18:00-20:00' when coll_time between 2000 and 2200 then '20:00-22:00' when coll_time between 2200 and 2400 then '22:00-00:00' end as coll_time from COLLISONS_L where coll_time is not null")

In [11]:
from pyspark.sql.functions import col
display(time_data.groupby('coll_time').count().sort(col('coll_time')))

coll for different days of the week

In [13]:

display(sqlContext.sql("SELECT count(*),day_of_week as year from COLLISONS_L group by day_of_week"))


number of alchohol cases involved for year

In [15]:
display(sqlContext.sql("SELECT count(*),acc_year as year from COLLISONS_L where alchohol_ivolved='Y' group by acc_year"))


no of coll for each day when alochol is involved

In [17]:
display(sqlContext.sql("SELECT count(*),day_of_week as day from COLLISONS_L where alchohol_ivolved='Y' group by day_of_week"))


In [18]:
import math

def distance(lat1,lon1):
   try:
   
    lat2 = 34.0654
    lon2 = -118.1721
    radius_in_miles = 3965

    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius_in_miles * c

    return d
   except:
    return 0.0

number of accidents near CSU LA

In [20]:
lat_lon = sqlContext.sql("SELECT case_id,CAST(LON as float),CAST(LAT as float) FROM COLLISONS_L");

In [21]:
distance_data=lat_lon.map(lambda (x,y,z):(x,distance(y,z)))

In [22]:
distance_df=distance_data.toDF(['id','distance'])

In [23]:
distance_df.registerTempTable("distance_table")

In [24]:
distance_data=sqlContext.sql("select case when distance between 0 and 5 then '0-5' when distance between 5 and 10 then '5-10' when distance between 10 and 15 then '10-15' when distance between 15 and 20 then '15-20' when distance between 20 and 25 then '20-25' when distance between 25 and 30 then '25-30' else '>30' end as distance from distance_table where distance is not null")

In [25]:
from pyspark.sql.functions import col
display(distance_data.groupby('distance').count().sort(col('distance')))

map data under 5 miles from CSULA

In [27]:
distance_data_map=lat_lon.map(lambda (x,y,z):(x,distance(y,z),y,z))

In [28]:
distance_data_map_csula=distance_data_map.toDF(['id','distance','lon','lat'])

In [29]:
distance_data_map_csula.registerTempTable("distance_table_map_csula")

In [30]:
display(sqlContext.sql("SELECT * FROM distance_table_map_csula where distance < 5"))

In [31]:
/FileStore/tables/sip0g0jm1469246874616/Victim_Tables__Collisions_20092013_SWITRS.csv


In [32]:
%sql
CREATE TABLE COLLISONS_VICTIM(
    object_id int,  
  case_id int,  
  party_numb int,  
  victim_role string,  
  victim_sex string,  
  victim_age int,  
  victim_deg_inj int,  
  victim_seating_pos String,  
  victim_safety_equip1 String,  
  victim_safety_equip2 String, 
 victim_ejected string, 
  accident_year string
 )
USING com.databricks.spark.csv
OPTIONS (path "/FileStore/tables/sip0g0jm1469246874616/Victim_Tables__Collisions_20092013_SWITRS.csv", header "true")

Victim Sex

In [34]:
display(sqlContext.sql("SELECT count(*) as total,victim_sex from COLLISONS_VICTIM group by victim_sex"))

Victim Severity

In [36]:
display(sqlContext.sql("SELECT count(*) as total,victim_deg_inj from COLLISONS_VICTIM group by victim_deg_inj"))

In [37]:
victim_data=sqlContext.sql("select case when victim_age between 0 and 10 then '0-10' when victim_age between 10 and 20 then '10-20' when victim_age between 20 and 30 then '20-30' when victim_age between 30 and 40 then '30-40' when victim_age between 40 and 50 then '40-50' when victim_age between 50 and 60 then '50-60' when victim_age between 60 and 70 then '60-70' else '>70' end as victim_age from COLLISONS_VICTIM where victim_age is not null")

In [38]:
from pyspark.sql.functions import col
display(victim_data.groupby('victim_age').count().sort(col('victim_age')))

In [39]:
/FileStore/tables/8tgkixcf1469248660605/Party_Tables__Collisions_20092013_SWITRS.csv


Party Table

In [41]:
%sql
DROP TABLE IF EXISTS COLLISONS_PARTY;

In [42]:
%sql
DROP TABLE IF EXISTS COLLISONS_PARTY;
CREATE TABLE COLLISONS_PARTY(
  object_id int,  
  case_id int,  
  party_numb int,  
  party_type string,  
  at_fault string,  
  party_sex string,  
  party_age int,  
  party_sobrity String,  
  party_drug_physical String,  
  direction_of_travel String, 
 party_safety_equip1 string, 
  party_safety_equip2 string,
  finance_response string,
  sp_info1 string,
  sp_info2 string,
  sp_info3 string,
  qaf_violation_code string,
  qaf_violation_cat string,
    qaf_violation_section string,
  qaf_violation_suffix string,
  qaf12 string,
party_numb_killed string,
  party_numb_injured string,
move_per_acc string,
  vech_year string,
  vech_make string,
  sdwd_vech_type string,
  chp_vech_type_towing string,
    chp_vech_type_towed string,
race string,
  in_attention string,
  special_info_f string,
    special_info_g string,
acc_year string
 )
USING com.databricks.spark.csv
OPTIONS (path "/FileStore/tables/8tgkixcf1469248660605/Party_Tables__Collisions_20092013_SWITRS.csv", header "true")

Party Sex

In [44]:
display(sqlContext.sql("SELECT count(*) as total,party_sex from COLLISONS_PARTY group by party_sex"))

Party age

In [46]:
party_data=sqlContext.sql("select case when party_age between 0 and 10 then '0-10' when party_age between 10 and 20 then '10-20' when party_age between 20 and 30 then '20-30' when party_age between 30 and 40 then '30-40' when party_age between 40 and 50 then '40-50' when party_age between 50 and 60 then '50-60' when party_age between 60 and 70 then '60-70' else '>70' end as party_age from COLLISONS_PARTY where party_age is not null")

In [47]:
from pyspark.sql.functions import col
display(party_data.groupby('party_age').count().sort(col('party_age')))

In [48]:
display(sqlContext.sql("SELECT count(*) as total,sdwd_vech_type from COLLISONS_PARTY group by sdwd_vech_type"))

In [49]:
display(sqlContext.sql("SELECT count(*) as total,vech_make from COLLISONS_PARTY group by vech_make"))